### Simple Tensors

In [26]:
import torch
a = torch.ones(3)
a

tensor([1., 1., 1.])

In [27]:
a[1]

tensor(1.)

In [28]:
a[2] = 2.0
a

tensor([1., 1., 2.])

In [29]:
points = torch.tensor([4.0, 1.0, 5.0,
                       3.0, 2.0, 1.0])
points

tensor([4., 1., 5., 3., 2., 1.])

In [30]:
points = points.reshape((3, 2))
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [31]:
points.shape

torch.Size([3, 2])

In [32]:
points[0]

tensor([4., 1.])

### 3D to grayscale

In [33]:
img_t = torch.randn(3, 5, 5)    # channels, rows, columns
# weights = torch.tensor([0.2126, 0.7152, 0.0722])
weights = torch.tensor([2., 5., -1.])

batch_t = torch.randn(2, 3, 5, 5)   # batch, channels, r, c

In [34]:
img_t

tensor([[[-2.2057, -0.4251,  1.5755,  0.7783, -0.1837],
         [-0.3726, -1.4847,  1.7220,  0.1103, -0.8804],
         [-0.3352, -0.4661, -0.8777,  0.4533,  2.1161],
         [-0.5476, -0.1672, -0.7649, -0.7134, -0.4763],
         [ 0.0652,  2.3634,  1.6161, -0.0697, -2.4253]],

        [[ 0.0094,  0.7417, -0.0292,  0.6128, -0.2286],
         [-0.4663, -0.2894,  0.2706, -1.6317, -0.6076],
         [ 0.5870, -1.1480,  1.2009,  1.2629, -1.1929],
         [-0.7825,  2.7406, -0.4501,  1.1848,  1.1377],
         [-0.1734,  0.6727,  0.3976,  2.4003,  0.4096]],

        [[ 0.3575,  0.6114, -1.7872, -0.7308, -0.3992],
         [ 0.5640, -0.0239,  0.5490,  1.8182,  1.9077],
         [-0.8026,  1.1296,  0.4353,  0.8978,  2.2192],
         [ 0.6991, -0.6158, -0.6072,  0.2651, -0.2604],
         [-0.6455, -1.0722, -0.5611,  0.1236, -0.5031]]])

In [35]:
img_gray_naive = img_t.mean(-3)     # Third from the end is channels
batch_gray_naive = batch_t.mean(-3) # It doesn't matter butch or one copy
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [36]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze_(-1)
img_weights = img_t * unsqueezed_weights
batch_weights = batch_t * unsqueezed_weights
img_gray_weighted = img_weights.sum(-3)
batch_gray_weighted = batch_weights.sum(-3)
batch_weights.shape, batch_t.shape, unsqueezed_weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

In [37]:
weights.unsqueeze(-1)

tensor([[ 2.],
        [ 5.],
        [-1.]])

In [38]:
unsqueezed_weights

tensor([[[ 2.]],

        [[ 5.]],

        [[-1.]]])

In [46]:
img_weights[0]

tensor([[-4.4115, -0.8502,  3.1510,  1.5565, -0.3675],
        [-0.7453, -2.9694,  3.4440,  0.2206, -1.7607],
        [-0.6704, -0.9322, -1.7554,  0.9066,  4.2323],
        [-1.0952, -0.3344, -1.5298, -1.4267, -0.9527],
        [ 0.1305,  4.7268,  3.2322, -0.1393, -4.8507]])

In [47]:
img_t[0]

tensor([[-2.2057, -0.4251,  1.5755,  0.7783, -0.1837],
        [-0.3726, -1.4847,  1.7220,  0.1103, -0.8804],
        [-0.3352, -0.4661, -0.8777,  0.4533,  2.1161],
        [-0.5476, -0.1672, -0.7649, -0.7134, -0.4763],
        [ 0.0652,  2.3634,  1.6161, -0.0697, -2.4253]])

In [54]:
(img_weights[0] == img_t[0] * 2).all().item()

True

In [40]:
img_gray_weighted_fancy = torch.einsum('...chw,c->...hw', img_t, weights)
batch_gray_weighted_fancy = torch.einsum('...chw,c->...hw', batch_t, weights)
batch_gray_weighted_fancy.shape

torch.Size([2, 5, 5])

### Names

In [41]:
weights_named = torch.tensor([0.2126, 0.7152, 0.0722],
                             names=['channels'])
weights_named

tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [42]:
img_named = img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print('img named:', img_named.shape, img_named.names)
print('batch named:', batch_named.shape, batch_named.names)

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


In [43]:
weights_aligned = weights_named.align_as(img_named)
weights_aligned.shape, weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

In [44]:
gray_named = (img_named * weights_aligned).sum('channels')
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

In [45]:
gray_plain = gray_named.rename(None)
gray_plain.shape, gray_plain.names

(torch.Size([5, 5]), (None, None))

### Types of Data

In [57]:
print(torch.float32 == torch.float)
print(torch.float64 == torch.double)
print(torch.float16 == torch.half)
print(torch.int8)
print(torch.uint8)
print(torch.int16 == torch.short)
print(torch.int32 == torch.int)
print(torch.int64 == torch.long)
print(torch.bool)

True
True
True
torch.int8
torch.uint8
True
True
True
torch.bool


The default is `torch.float32` (just `torch.float`)

In [58]:
double_points = torch.ones(10, 2, dtype=torch.double)
short_points = torch.tensor([[1, 2], [3, 4]], dtype=torch.short)
double_points.dtype, short_points.dtype

(torch.float64, torch.int16)

In [59]:
double_points = torch.zeros(10, 2).to(torch.double)
double_points.dtype

torch.float64

In [60]:
points_64 = torch.rand(5, dtype=torch.double)
points_short = points_64.to(torch.short)
points_64 * points_short

tensor([0., 0., 0., 0., 0.], dtype=torch.float64)

In [61]:
points_64

tensor([0.0270, 0.6843, 0.7799, 0.3366, 0.3057], dtype=torch.float64)

In [62]:
points_short

tensor([0, 0, 0, 0, 0], dtype=torch.int16)

### Other operations

In [63]:
a = torch.ones(3, 2)
a_t = torch.transpose(a, 0, 1)
a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

In [64]:
a = torch.ones(3, 2)
a_t = a.transpose(0, 1)
a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

In [108]:
a = torch.tensor([[45, 90], [30, 60]])
torch.cos(a)

tensor([[ 0.5253, -0.4481],
        [ 0.1543, -0.9524]])

### Storage

In [67]:
points = torch.tensor([[4.0, 1.0],
                       [5.0, 3.0],
                       [2.0, 1.0]])
points.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]

In [79]:
points[2][0].item() == points.storage()[4]

True

In [80]:
points.zero_()
points

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

The resulting tensor has offset 2 in the storage (since we need to skip the first point, which has two items)

In [81]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]
second_point.storage_offset()

2

The stride is a tuple indicating the number of elements in the storage that have to be skipped when the index is increased by 1 in each dimension

In [82]:
points.stride()

(2, 1)

In [84]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1].clone()    # For copying
second_point[0] = 10.0
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

Shorthand transpose

In [85]:
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [86]:
points.stride()

(2, 1)

In [87]:
points_t.stride()

(1, 2)

This tells us that increasing the first index by one in points—for example, going from `points[0,0]` to `points[1,0]` — will skip along the storage by two elements, while increasing the second index—from `points[0,0]` to `points[0,1]` — will skip along the storage by one.

### Transposing in higher dimensions

In [89]:
some_t = torch.ones(3, 4, 5)
transpose_t = some_t.transpose(0, 2)
transpose_t.shape

torch.Size([5, 4, 3])

In [92]:
some_t.stride(), transpose_t.stride()

((20, 5, 1), (1, 5, 20))

In [93]:
points.is_contiguous(), points_t.is_contiguous()

(True, False)

In [94]:
points.stride(), points_t.stride()

((2, 1), (1, 2))

In [95]:
points_t_cont = points_t.contiguous()
points_t_cont

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [96]:
points_t_cont.stride()

(3, 1)

In [98]:
# points_gpu = points.cuda()
# points_gpu = points.cuda(0)
# points_cpu = points_gpu.cpu()

In [99]:
points = torch.ones(3, 4)
points_np = points.numpy()
points_np

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)

### Saving & Loading

In [104]:
torch.save(points,
           '../DL_with_PyTorch-Eli_Stevens/data/ourpoints.t')

In [105]:
points = torch.load('../DL_with_PyTorch-Eli_Stevens/data/ourpoints.t')
points

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

In [106]:
import h5py

f = h5py.File('../DL_with_PyTorch-Eli_Stevens/data/ourpoints.hdf5', 'w')
dset = f.create_dataset('coords', data=points.numpy())
f.close()

In [107]:
f = h5py.File('../DL_with_PyTorch-Eli_Stevens/data/ourpoints.hdf5', 'r')
dset = f['coords']
last_points = torch.from_numpy(dset[-2:])
f.close()